# <div style="background:rgba(53, 144, 235, 0.64); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> 📊 One-Hot Encoding: Convirtiendo Categorías en Números
</div>

<div style="background:#E8F0FE; border-left: 5px solid rgba(53, 144, 235, 0.64); padding: 12px; margin-bottom: 16px; color:#181818;">
El preprocesamiento de datos es un paso crucial en cualquier proyecto de Machine Learning. La transformación adecuada de variables categóricas puede marcar la diferencia entre un modelo que funciona y uno que no.
</div>

**¡Bienvenido!** En esta presentación se abordará:

1. ¿Qué es One-Hot Encoding?
2. Ejemplo Práctico.
3. ¿Cuándo NO usar One-Hot Encoding?
4. One-Hot Encoding por Frecuencias.
5. Ordinal Encoder.
6. Count or Frequency Encoding.

Referencia: capítulo _Encoding Categorical Variables_ del libro _Python Feature Engineering Cookbook_ de Soledad Galli.

### **Importación de Librerías**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

### **Dataset Utilizado**

Se utiliza un dataset sobre comida India obtenido de Kaggle.

In [2]:
df = pd.read_csv(
    "../../../datasets/Alumnos/lambda-09-indian_food_dataset/data/indian_food.csv"
)

X = df.copy()

# Mirar los primeros registros
X.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


## <div style="background:rgba(99, 102, 241, 0.65); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> 💡 1. ¿Qué es One-Hot Encoding?
</div>

En el preprocesamiento de datos, **One-Hot Encoding** es una técnica para **transformar variables categóricas** en un formato numérico que los modelos de Machine Learning puedan entender y utilizar.

* **Representación Binaria:** Cada **categoría única** dentro de una variable se convierte en una **nueva columna del dataset**.
* **Valor:** Esta nueva columna toma un valor de **1** si la categoría está presente para esa observación, y **0** si no lo está.

<div style="background: #FEF3C7; border-left: 5px solid #F59E0B; padding: 12px; margin-top: 12px; margin-bottom: 12px; color: #181818;">En esencia, se transforma una columna categórica en múltiples columnas, donde solo una de ellas tendrá el valor 1 y el resto tendrá valor 0.
</div>

Características: 

* **Apto para Modelos Lineales:** One-Hot Encoding transforma categorías nominales en una representación numérica que modelos como la Regresión Lineal o Logística pueden procesar eficazmente.

* **Expande el Espacio de Características:** el proceso aumenta el número de características (columnas) en el *dataset*, lo que se conoce como **expansión de dimensionalidad**.

## <div style="background:rgba(34, 197, 94, 0.65); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> 🚀 2. Ejemplo Práctico
</div>

Seguiremos los siguientes pasos:

1. **Preparar los datos**: copiar y dividir el dataset en train/test para evitar data leakage.
2. **Seleccionar las variables categóricas**: identificar qué columnas necesitan ser transformadas.
3. **Crear el encoder**: instanciar `OneHotEncoder` con los parámetros deseados.
4. **Crear el ColumnTransformer**: envolver el encoder en un `ColumnTransformer`.
5. **Ajustar (fit)**: entrenar el encoder con los datos de **entrenamiento** únicamente.
6. **Transformar**: aplicar la transformación tanto al conjunto de entrenamiento como al de prueba usando el encoder ya ajustado.

In [3]:
# Dividir en train/test para evitar data leakage (una buena practica que aplica el autor)
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Seleccionar las variables categoricas a encodear
cat_vars = ["diet"]

In [4]:
# OneHotEncoder para k-1 variables
encoder = OneHotEncoder(drop=None, sparse_output=False)

ct = ColumnTransformer(
    [("encoder", encoder, cat_vars)], remainder="passthrough"
).set_output(transform="pandas")

ct.fit(X_train)

# Mostrar las categorías detectadas
print("Categorías detectadas por el encoder:")
for col, cats in zip(cat_vars, ct.named_transformers_["encoder"].categories_):
    print(f"   Variable: {col}")
    print(f"   Categorías: {', '.join(cats)}")
    print(f"   Total: {len(cats)} categorías")

X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

Categorías detectadas por el encoder:
   Variable: diet
   Categorías: non vegetarian, vegetarian
   Total: 2 categorías


In [5]:
print("\nDataset transformado (train):")

# Comparación: columna original 'diet' vs columnas generadas por el encoder
comparison_df = pd.DataFrame(
    {
        "diet_original": X_train.iloc[9:14]["diet"].values,
        "diet_non_vegetarian": X_train_enc.iloc[9:14][
            "encoder__diet_non vegetarian"
        ].values,
        "diet_vegetarian": X_train_enc.iloc[9:14]["encoder__diet_vegetarian"].values,
    }
)

comparison_df.head()


Dataset transformado (train):


,diet_original,diet_non_vegetarian,diet_vegetarian
0,vegetarian,0.0,1.0
1,non vegetarian,1.0,0.0
2,vegetarian,0.0,1.0
3,vegetarian,0.0,1.0
4,vegetarian,0.0,1.0


In [6]:
print("\nDataset transformado (test):")

# Comparación: columna original 'diet' vs columnas generadas por el encoder
comparison_df = pd.DataFrame(
    {
        "diet_original": X_test.iloc[9:14]["diet"].values,
        "diet_non_vegetarian": X_test_enc.iloc[9:14][
            "encoder__diet_non vegetarian"
        ].values,
        "diet_vegetarian": X_test_enc.iloc[9:14]["encoder__diet_vegetarian"].values,
    }
)

comparison_df.head()


Dataset transformado (test):


,diet_original,diet_non_vegetarian,diet_vegetarian
0,vegetarian,0.0,1.0
1,vegetarian,0.0,1.0
2,vegetarian,0.0,1.0
3,non vegetarian,1.0,0.0
4,vegetarian,0.0,1.0


## <div style="background:rgba(239, 68, 68, 0.65); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> ⚠️ 3. ¿Cuándo NO usar One-Hot Encoding?
</div>

Existe el riesgo de la **explosión de dimensionalidad** y la ineficiencia que eso conlleva.

- **Alta Cardinalidad**: se crearán muchas columnas si la variable tiene **demasiadas categorías únicas**.

- **Variables Ordinales**: one-hot encoding ignora el orden natural de las variables categóricas (ej: 'Bajo', 'Medio', 'Alto').

- **Rendimiento**: un gran número de columnas puede aumentar el consumo de memoria durante el entrenamiento.

A continuación veremos ejemplos de estos casos:

### **Ejemplo 1: Alta Cardinalidad - Variable `region`**

La variable `region` tiene muchas categorías únicas (diferentes regiones de India), lo que genera demasiadas columnas:

In [7]:
# Ejemplo de alta cardinalidad con la variable 'region'
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# La variable 'region' tiene muchas categorías únicas
print(f"\nRegiones: {X_train['region'].unique()}")

# Aplicar One-Hot Encoding
cat_vars_region = ["region"]
encoder = OneHotEncoder(
    drop="first", sparse_output=False, handle_unknown="infrequent_if_exist"
)

ct = ColumnTransformer(
    [("encoder", encoder, cat_vars_region)], remainder="passthrough"
).set_output(transform="pandas")

ct.fit(X_train)
X_train_enc_region = ct.transform(X_train)

# Mostrar solo las columnas generadas por el encoder
region_cols = [
    col for col in X_train_enc_region.columns if col.startswith("encoder__region_")
]
X_train_enc_region[region_cols].head()


Regiones: ['South' 'East' 'West' 'North' '-1' 'North East' 'Central' nan]


,encoder__region_Central,encoder__region_East,encoder__region_North,encoder__region_North East,encoder__region_South,encoder__region_West,encoder__region_nan
38,0.0,0.0,0.0,0.0,1.0,0.0,0.0
143,0.0,0.0,0.0,0.0,1.0,0.0,0.0
84,0.0,1.0,0.0,0.0,0.0,0.0,0.0
55,0.0,0.0,0.0,0.0,0.0,1.0,0.0
219,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### **Ejemplo 2: Combinación de Variables categóricas**

Cuando combinas múltiples variables categóricas, el número de columnas crece exponencialmente:

In [8]:
# Ejemplo de multiples variables combinadas
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Codificar múltiples variables al mismo tiempo
cat_vars_multiple = ["course", "region"]

encoder_full = OneHotEncoder(drop=None, sparse_output=False)

ct_multiple = ColumnTransformer(
    [("encoder", encoder_full, cat_vars_multiple)], remainder="passthrough"
).set_output(transform="pandas")

ct_multiple.fit(X_train)
X_train_multiple = ct_multiple.transform(X_train)

# Mostrar solo las columnas del encoder
encoder_cols = [col for col in X_train_multiple.columns if col.startswith("encoder__")]

X_train_multiple[encoder_cols].head()

,encoder__course_dessert,encoder__course_main course,encoder__course_snack,encoder__course_starter,encoder__region_-1,encoder__region_Central,encoder__region_East,encoder__region_North,encoder__region_North East,encoder__region_South,encoder__region_West,encoder__region_nan
38,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
143,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
84,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
55,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
219,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## <div style="background:rgba(245, 158, 11, 0.65); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> 🎯 4. One-Hot Encoding por Frecuencias
</div>

Una forma práctica de evitar la explosión de dimensionalidad es codificar sólo las categorías más frecuentes y agrupar las demás.

* **Codificación Selectiva:** se crean nuevas features únicamente con las regiones más frecuentes.

* **Agrupación de Categorías Raras:** se crea una columna adicional (`infrequent_sklearn`) donde se agrupan las categorías menos frecuentes.

* **Reducción de Dimensionalidad:** esto conserva la información de las categorías importantes y reduce significativamente el número de columnas creadas por el encoder.

In [9]:
# Ejemplo One-Hot por frecuencias

# Split para evitar data leakage
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

X_train["region"].value_counts(dropna=False).sort_values(ascending=False).reset_index()

,region,count
0,West,56
1,South,51
2,North,42
3,East,25
4,North East,18
5,-1,8
6,Central,3
7,NaN,1


In [10]:
encoder = OneHotEncoder(
    min_frequency=20,  # cantidad de ocurrrencias minimas
    max_categories=5,
    sparse_output=False,
).set_output(transform="pandas")

X_train_enc = encoder.fit_transform(X_train[["region"]])
X_test_enc = encoder.transform(X_test[["region"]])

# Agregar la columna region original para comparación al principio
X_test_enc.insert(0, "region_original", X_test["region"].values)
X_test_enc.head()

,region_original,region_East,region_North,region_South,region_West,region_infrequent_sklearn
190,West,0.0,0.0,0.0,1.0,0.0
6,North,0.0,1.0,0.0,0.0,0.0
79,East,1.0,0.0,0.0,0.0,0.0
205,West,0.0,0.0,0.0,1.0,0.0
117,-1,0.0,0.0,0.0,0.0,1.0


## <div style="background:rgba(14, 165, 233, 0.65); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> 🔢 5. Ordinal Encoder
</div>

El OrdinalEncoder asigna valores numéricos a categorías siguiendo un orden predefinido.

* **Orden Jerárquico:** es útil cuando la variable tiene un **sentido de orden natural** (por ejemplo, 'Bajo' < 'Medio' < 'Alto').

* **Preserva Relaciones:** a diferencia del One-Hot Encoding, el Ordinal Encoder **mantiene la relación ordinal** entre las categorías.

* **Asignación Numérica:** cada categoría se mapea a un número entero según el orden especificado (0, 1, 2, etc.).

In [11]:
# Ejemplo OrdinalEncoder

# Definir el orden jerárquico de las categorías
order = ["starter", "snack", "main course", "dessert"]

# Crear y aplicar el encoder
categories = [order]
enc = OrdinalEncoder(
    categories=categories,
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    encoded_missing_value=-1,
).set_output(transform="pandas")

X_enc = enc.fit_transform(X[["course"]])

# Mostrar el mapeo de categorías a valores ordinales
print("Mapeo de categorías a valores ordinales:")
mapeo_df = pd.DataFrame({"Categoría": order, "Valor Ordinal": range(len(order))})
mapeo_df.head()

Mapeo de categorías a valores ordinales:


,Categoría,Valor Ordinal
0,starter,0
1,snack,1
2,main course,2
3,dessert,3


In [12]:
# Ejemplo con el dataset de entrenamiento mostrando la transformación

# Dividir en train/test
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Aplicar el encoder al conjunto de entrenamiento
X_train_enc = enc.fit_transform(X_train[["course"]])

# Crear DataFrame de comparación con algunas filas del conjunto de entrenamiento
comparison_df = pd.DataFrame(
    {
        "course_original": X_train.iloc[0:10]["course"].values,
        "course_encoded": X_train_enc.iloc[0:10]["course"].values,
    }
)

comparison_df.head()

,course_original,course_encoded
0,dessert,3.0
1,main course,2.0
2,main course,2.0
3,dessert,3.0
4,dessert,3.0


## <div style="background:rgba(236, 72, 153, 0.65); padding: 18px; border-radius: 12px; font-size: 0.8em; margin-bottom: 16px; color: #ffffffff; font-weight: bold;"> 📊 6. Count or Frequency Encoding
</div>

El Count o Frequency Encoding reemplaza las categorías con el conteo o la fracción de observaciones que muestran esa categoría.

* **Count Encoding:** reemplaza cada categoría con el **número de veces** que aparece en el dataset.

* **Frequency Encoding:** reemplaza cada categoría con la **frecuencia relativa** de observaciones.

* **Utilidad:** estos métodos son especialmente útiles cuando existe una **relación entre la frecuencia de la categoría y el objetivo**. Por ejemplo, en ventas, la frecuencia de un producto puede indicar su popularidad.

* **Limitación:** si dos categorías diferentes tienen el **mismo número de observaciones**, serán reemplazadas por el mismo valor, lo que puede llevar a **pérdida de información**.

In [13]:
# Count Encoding

# Dividir en train/test
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Calcular el conteo de cada región en el conjunto de entrenamiento
region_counts = X_train["region"].value_counts().to_dict()

# Aplicar Count Encoding
X_train_count = X_train.copy()
X_train_count["region_count"] = X_train_count["region"].map(region_counts)

print("Count Encoding:")
comparison_count = (
    X_train_count[["region", "region_count"]]
    .drop_duplicates()
    .sort_values("region_count", ascending=False)
)
comparison_count.head(7)

Count Encoding:


,region,region_count
55,West,56.0
38,South,51.0
104,North,42.0
84,East,25.0
245,North East,18.0
248,-1,8.0
253,Central,3.0


In [14]:
# Frequency Encoding

# Calcular la frecuencia de cada región (conteo / total)
region_frequency = X_train["region"].value_counts(normalize=True).to_dict()

# Aplicar Frequency Encoding
X_train_freq = X_train.copy()
X_train_freq["region_frequency"] = X_train_freq["region"].map(region_frequency)

# Mostrar
print("Frequency Encoding:")
comparison_freq = (
    X_train_freq[["region", "region_frequency"]]
    .drop_duplicates()
    .sort_values("region_frequency", ascending=False)
)
comparison_freq.head(7)

Frequency Encoding:


,region,region_frequency
55,West,0.275862
38,South,0.251232
104,North,0.206897
84,East,0.123153
245,North East,0.088670
248,-1,0.039409
253,Central,0.014778


In [15]:
# Ejemplo con el conjunto de entrenamiento

# Mostrar las primeras filas del conjunto de entrenamiento con ambos encodings
comparison_df = pd.DataFrame(
    {
        "region_original": X_train.iloc[0:10]["region"].values,
        "count_encoding": X_train_count.iloc[0:10]["region_count"].values,
        "frequency_encoding": X_train_freq.iloc[0:10]["region_frequency"].values,
    }
)

comparison_df.head()

,region_original,count_encoding,frequency_encoding
0,South,51.0,0.251232
1,South,51.0,0.251232
2,East,25.0,0.123153
3,West,56.0,0.275862
4,West,56.0,0.275862
